# Ensemble Evaluation: Timepoint 1

Location: New York State

Timepoint 1: April 3, 2020. Setting: New York State at the beginning of the pandemic when masking was the main preventative measure. No vaccines available. Reinfection not considered.

## Set up for ensemble modeling

### Load dependencies
Import functionality from the pyciemss library to allow for model sampling and calibration.

In [1]:
import os
import pandas as pd
import numpy as np
from pyciemss.Ensemble.interfaces import (
    load_and_sample_petri_ensemble, load_and_calibrate_and_sample_ensemble_model
)
from pyciemss.PetriNetODE.interfaces import (
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model,
    load_and_optimize_and_sample_petri_model,
    load_and_calibrate_and_optimize_and_sample_petri_model
)
from pyciemss.visuals import plots

### Collect relevant models
<!-- We have chosen x number of models to capture the relevant COVID-19 dynamics for this setting. 
 - `model1` contains compartments SEIRHD, and is stratified by age into four groups.
 - `model2` is the same as `model1`, but allows for reinfection
 - `model3` is the same as `model1`, but with a variation in transmission rate to account for masking efficacy and compliance. -->

In [36]:
model1_location = "../../notebook/ensemble_eval_sa/operative_models/SEIRHD_base_model_ee.json"
model2_location = "../../notebook/ensemble_eval_sa/operative_models/SEIRHD_npi1_ee.json"
model3_location = "../../notebook/ensemble_eval_sa/operative_models/SEIRHD_npi1_beta_c_varying_ee.json"
model4_location = "../../notebook/ensemble_eval_sa/operative_models/SEIRHD_npi1_k_varying_ee.json"
model5_location = "../../notebook/ensemble_eval_sa/operative_models/SEIRHD_npi1_age_stratified_v1.json"
model6_location = "../../notebook/ensemble_eval_sa/operative_models/SEIRHD_npi1_age_stratified_v2.json"
model7_location = "../../notebook/ensemble_eval_sa/operative_models/SEIRHD_npi1_age_stratified_v3.json"
model8_location = "../../notebook/ensemble_eval_sa/operative_models/SEIRD_ymo_age_strat.json"

### Gather data, and set training and forecast intervals
For this timepoint, only case and death data is available for calibration. 

We take the total population of New York State to be 19,340,000. Population age-structure estimates for New York State were taken (and scaled appropriately) from [here](https://www.health.ny.gov/statistics/vital_statistics/2016/table01.htm).

First recorded case in New York State: March 1, 2020.

Also relevant is that a statewide stay-at-home order for non-essential workers was implemented on March 22, 2020, and masking policy was implemented on April 15, 2020. A nice list of COVID-19 policy interventions for New York City and State has been compiled [here](https://www.investopedia.com/historical-timeline-of-covid-19-in-new-york-city-5071986![image.png](attachment:d380d245-11ca-41f1-9863-e2a2dcf5ebce.png)![image.png](attachment:b30f6e31-bc7f-43a8-b7c8-b1df78db3b86.png)![image.png](attachment:b5109588-59ed-447c-9271-50eb56fe9081.png)![image.png](attachment:2c42138a-696d-4cef-9b79-4843d38ff029.png)).

In [10]:
url = 'https://raw.githubusercontent.com/DARPA-ASKEM/experiments/main/thin-thread-examples/milestone_12month/evaluation/ensemble_eval_SA/datasets/aabb3684-a7ea-4f60-98f1-a8e673ad6df5/dataset.csv'
ny_data = pd.read_csv(url)

# Grab test data for four-week forecast (04/03/2020 - 05/01/2020)
test_data = ny_data[41:101].reset_index()
test_data = test_data.drop(columns="timestep")
test_data = test_data.drop(columns="index")

# Select historical data from 03/01/2020 up to Timepoint 1, 04/02/2020 (the first 73 rows)
# No hospitalization data at this point
ny_data = ny_data[41:72].reset_index(drop=True)
ny_data1 = ny_data.assign(timepoints=[float(i) for i in range(len(ny_data))])
ny_data = ny_data1[["timepoints", "I", "H", "D"]]
ny_data[["I", "H", "D"]].to_csv("NY_data1.csv")

# Set timepoints
start_timepoint = 0
stop_timepoint = len(ny_data) + 28 # simulate for four weeks after end of data
timepoints = [float(i) for i in range(stop_timepoint + 1)]

# Calibrate and sample an ensemble of one model

In [156]:
num_samples = 100
model_paths = [model2_location]
data_path = "../../notebook/ensemble_eval_sa/NY_data1.csv"
weights = [1]
solution_mappings = [{"I": "I", "H": "H", "D": "D"}]
# solution_mappings = [{"I": "infected", "H": "hospitalized", "D": "dead"}]
# timepoints = timepoints[0:30]
# Run the calibration and sampling
result1 = load_and_calibrate_and_sample_ensemble_model(
    model_paths,
    data_path,
    weights,
    solution_mappings,
    num_samples,
    timepoints,
    verbose=True,
    total_population=19340000,
    num_iterations=250,
    time_unit="days",
    visual_options={"title": "Calibrated Ensemble", "subset":".*_sol"}
)

# Save results
result1["data"].to_csv(
    "partI_ensemble_of_one_results.csv", index=False
)
result1["quantiles"].to_csv(
    "partI_ensemble_of_one_quantiles.csv", index=False
)

# Plot results
schema = plots.trajectories(pd.DataFrame(result1["data"]), subset=".*_sol",
                            points=test_data.reset_index(drop=True).rename(columns={"I":"I_data", "H":"H_data", "D":"D_data"}),
                           )
schema = plots.pad(schema, 5)
plots.ipy_display(schema)

iteration 0: loss = 700.9401803314686
iteration 25: loss = 673.134889036417
iteration 50: loss = 637.5662955343723
iteration 75: loss = 634.1837383806705
iteration 100: loss = 635.7135492861271
iteration 125: loss = 634.2485391199589
iteration 150: loss = 635.6275178492069
iteration 175: loss = 634.7659775316715
iteration 200: loss = 641.5253984034061
iteration 225: loss = 636.9702662527561



In [159]:
num_samples = 100
model_paths = [model6_location]
data_path = "../../notebook/ensemble_eval_sa/NY_data1.csv"
weights = [1]
# solution_mappings = [{"I": "I", "H": "H", "D": "D"}]
solution_mappings = [{"I": "infected", "H": "hospitalized", "D": "dead"}]

# Run the calibration and sampling
result2 = load_and_calibrate_and_sample_ensemble_model(
    model_paths,
    data_path,
    weights,
    solution_mappings,
    num_samples,
    timepoints,
    verbose=True,
    total_population=19340000,
    num_iterations=100,
    time_unit="days",
    visual_options={"title": "Calibrated Ensemble", "subset":".*_sol"}
)

# # Save results
# result2["data"].to_csv(
#     "partI_ensemble_of_one_results.csv", index=False
# )
# result2["quantiles"].to_csv(
#     "partI_ensemble_of_one_quantiles.csv", index=False
# )

# Plot results
schema = plots.trajectories(pd.DataFrame(result2["data"]), subset=".*_sol",
                            points=test_data.reset_index(drop=True).rename(columns={"I":"I_data", "H":"H_data", "D":"D_data"}),
                           )
schema = plots.pad(schema, 5)
plots.ipy_display(schema)

iteration 0: loss = 716.8114766180515
iteration 25: loss = 705.2775605022907
iteration 50: loss = 676.0982182323933
iteration 75: loss = 636.5155728161335



In [158]:
schema = plots.trajectories(pd.DataFrame(result2["data"]), subset=".*_sol",
                            points=test_data.reset_index(drop=True).rename(columns={"I":"I_data", "H":"H_data", "D":"D_data"}),
                           )
schema = plots.pad(schema, 5)
plots.ipy_display(schema)

In [ ]:
num_samples = 100
model_paths = [model7_location]
data_path = "../../notebook/ensemble_eval_sa/NY_data1.csv"
weights = [1]
# solution_mappings = [{"I": "I", "H": "H", "D": "D"}]
solution_mappings = [{"I": "infected", "H": "hospitalized", "D": "dead"}]

# Run the calibration and sampling
result3 = load_and_calibrate_and_sample_ensemble_model(
    model_paths,
    data_path,
    weights,
    solution_mappings,
    num_samples,
    timepoints,
    verbose=True,
    total_population=19340000,
    num_iterations=50,
    time_unit="days",
    visual_options={"title": "Calibrated Ensemble", "subset":".*_sol"}
)

# # Save results
# result3["data"].to_csv(
#     "partI_ensemble_of_one_results.csv", index=False
# )
# result3["quantiles"].to_csv(
#     "partI_ensemble_of_one_quantiles.csv", index=False
# )

# Plot results
schema = plots.trajectories(pd.DataFrame(result3["data"]), subset=".*_sol",
                            points=test_data.reset_index(drop=True).rename(columns={"I":"I_data", "H":"H_data", "D":"D_data"}),
                           )
schema = plots.pad(schema, 5)
plots.ipy_display(schema)

iteration 0: loss = 720.4351313114166
iteration 25: loss = 698.6325839757919


In [163]:
schema = plots.trajectories(pd.DataFrame(result3["data"]), subset=".*_sol",
                            points=test_data.reset_index(drop=True).rename(columns={"I":"I_data", "H":"H_data", "D":"D_data"}),
                           )
schema = plots.pad(schema, 5)
plots.ipy_display(schema)

# Calibrate and sample an ensemble of multiple models

In [ ]:
num_samples = 100
model_paths = [model2_location, model6_location, model7_location]
data_path = "../../notebook/ensemble_eval_sa/NY_data1.csv"
weights = [1/len(model_paths) for i in model_paths]
solution_mappings = [{"I": "I", "H": "H", "D": "D"},
                     {"I": "infected", "H": "hospitalized", "D": "dead"}, 
                     {"I": "infected", "H": "hospitalized", "D": "dead"}
                     ]

# Run the calibration and sampling
result4 = load_and_calibrate_and_sample_ensemble_model(
    model_paths,
    data_path,
    weights,
    solution_mappings,
    num_samples,
    timepoints,
    verbose=True,
    total_population=19340000,
    num_iterations=100,
    time_unit="days",
    visual_options={"title": "Calibrated Ensemble", "subset":".*_sol"}
)

# Save results
result4["data"].to_csv(
    "partI_ensemble_of_many_results.csv", index=False
)
result4["quantiles"].to_csv(
    "partI_ensemble_of_many_quantiles.csv", index=False
)

# Plot results
schema = plots.trajectories(pd.DataFrame(result4["data"]), subset=".*_sol",
                            points=test_data.reset_index(drop=True).rename(columns={"I":"I_data", "H":"H_data", "D":"D_data"}),
                           )
schema = plots.pad(schema, 5)
plots.ipy_display(schema)

In [ ]:
from pyciemss.utils.interface_utils import cdc_reformatcsv

q_ensemble_data = cdc_reformatcsv(filename="partI_ensemble_of_many_quantiles.csv", 
                                  solution_string_mapping={"Infected": "case", "Hospitalizations": "hosp.", "D": "death"}, 
                                  forecast_start_date="2023-08-03",
                                  location="US",
                                  drop_column_names=["timepoint_id", "number_days", "inc_cum", "output", "Forecast_Backcast"])
q_ensemble_data.to_csv(os.path.join(DEMO_PATH, "results_petri_ensemble/CDCformat_calibrated_quantile_results.csv"), index=False)